### Naive Bayes SPAM Filter

This task aimed to classify new emails as spam or ham using Bayes Theorem and a set of pre-classified emails. The Naive Bayes formula assumes that each element is independent, and in this case, it assesses each word as an individual based on its occurrence in the training data. This spam filter will not account for phrases common to spam emails. This makes the filter less accurate but is much more efficient as accounting for every combination of words that could be in an email would become exponentially more complex.<br>
Formula used:<br>
$P(email|S) = (P(S) * P(word1|S) * P(word2|S)) / (P(S) * P(word1|S) * P(word2|S) + P(H) * P(word1|H) * P(word2|H))$

Calculations for this task can be found at the end of this document.


In [90]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

In [91]:
def analyse_emails(emails):
    """
    AIM: examine a list of emails. Get the probability of each word in the list of emails occurring.
    ARGUMENTS: A list of strings representing emails.
    ACTION: Split the emails into a list of words.Get the probability of each word occurring.
    RETURN: A dictionary containing words and their probability of occurring and the total number of words in
            the emails.
    """
    words_in_emails = []
    
    # Loop iterates through all the emails.
    for email in emails:
        
        # Set all words to lower case to prevent duplicate words.
        email = email.lower()
        
        # Splits each email into a list of words and appends it to the list of all words.
        words_in_emails += email.split(" ")
        
    # Convert list to set so that there is a list of unique words.
    unique_words = set(words_in_emails)
    
    # Total number of words for use in calculations
    total_words = len(words_in_emails)
    
    probability_dict = {}
    
    # Iterates through the list of unique words.
    for word in unique_words:
        # Adds the word to a dictionary with the word as the key and the probability 
        # of that word occurring as the value.
        probability_dict[word] = (words_in_emails.count(word)) / (len(words_in_emails))
        
    
    probability_dict["total_words"] =  total_words
        
    return probability_dict

In [92]:
def prior_probability(probability_emails, rest_of_emails):
    """
    AIM: Gets the probability of the first set of emails passed to it. Used for getting P(S) and P(H).
    ARGUMENTS: Two lists of emails. 
    ACTION: Divides the number of emails in the first list by the total number of emails to get its probability.
    RETURN: The probability of the first list passed to it as an argument.
    """
    return len(probability_emails) / (len(probability_emails) + len(rest_of_emails))

In [93]:
def probability_of_A_given_B(new_email, previous_emails, previous_probability):
    """
    AIM: Calculates the probability P(A|B). Used to calculate P(email|Spam) and P(email|Ham).
    ARGUMENTS: email: The email to be assessed, previous_emails: The emails to compare against, 
               previous_probability: The probability P(S) or P(H). 
    ACTION: Compare email to training data to assess if spam or ham.
    RETURN: The probability that it is spam or ham.
    """
    
    # Gets probability data from training data
    probability_dict = analyse_emails(previous_emails)
    words_in_new_email = new_email.split(" ")
    
    # Adds the previous probability to calculation.
    probability = previous_probability
    
    # Iterates through words in the new email.
    for word in words_in_new_email:
        # Checks if the word exists in the training data returns 0 if it does not.
        mul = probability_dict.get(word, 0)
        
        # If the word is in the training data use it in the equation. If not use laplace smoothing.
        if mul > 0:
            probability *= mul
        else:
            # If not in training data use laplace smoothing.
            probability *= 1 / (probability_dict["total_words"] + 2)
        
    return probability

In [94]:
def get_percentage(probability_a, probability_b):
    """
    AIM: Take the probability of a and b and turns it into a more human-readable percentage. This is used to
         complete the bayes equation.   
    ARGUMENTS: probability_a: the probability that will be returned as a percentage.
               probability_b: The second probability used to complete the denominator.
    ACTION: Turns probability_a into a percentage.
    RETURN: an int between 0 and 100.
    """
    return round(((probability_a / (probability_a + probability_b)) * 100), 2)

In [95]:
def update_training_data(email, training_data, is_spam):
    """
    AIM: Adds email to the correct set of training data after it has been classified.
    ARGUMENTS: email: email that was classified, 
               training_data: the data set to be updated,
               is_spam: boolean value True when email is spam.
    ACTION: If is_spam is True the email is appended to training_data["spam"],
            If is_spam is False the email is appended to training_data["ham"].
    RETURN: A dictionary of the updated training data.
    """
    
    if is_spam:
        training_data["spam"].append(email)
    else:
        training_data["ham"].append(email)
        
    return training_data
        

In [96]:
def classify_email(email, spam, ham):
    """
    AIM: To classify an email as spam or ham.
    ARGUMENTS: email: email to be classified,
               spam: Spam training data,
               ham: Ham training data
    ACTION: Uses spam and ham training data to calculate if the email is spam or ham
    RETURN: A dictionary containing the probability of the email being ham or spam
            and a boolean which if True indicates that the email is spam.
    """
    
    # Calculates P(S)
    spam_prior_probability = prior_probability(spam, ham)
    # Calculates P(H)
    ham_prior_probability = prior_probability(ham, spam)
    
    # Get P(email|S)
    spam_test = probability_of_A_given_B(email, spam, spam_prior_probability)
    # Get P(email|H)
    ham_test = probability_of_A_given_B(email, ham, ham_prior_probability)
    
    # Converts P(email|S) to a percentage value.
    percentage_spam = get_percentage(spam_test, ham_test)
    # Converts P(email|H) to a percentage value.
    percentage_ham = get_percentage(ham_test, spam_test)
    # Tests if the email is more likely to be spam or ham.
    is_spam = spam_test > ham_test
    
    # Packs data into a dictionary.
    email_stats = {
        "spam": percentage_spam,
        "ham": percentage_ham,
        "is_spam": is_spam
    }
    
    return email_stats
    

In [97]:
# Initialise the training data or reset it for testing.
training_data = {}
training_data["spam"] = previous_spam.copy()
training_data["ham"] = previous_ham.copy()

In [98]:
# Copies all emails for testing into one list to make it easier to test.
all_emails = new_emails["spam"].copy() + new_emails["ham"].copy()

### Testing


The new emails were run through my classify_email function without updating the training data after classification. These results match the calculations carried out for this task which can be seen below in the calculations section. As predicted, one of the emails meant to be spam was misclassified as ham.<br> 
The emails were then run through the classify_email function, and the training data was updated after each classification. With the training data updating after each classification, fewer unknown words existed, and all emails were correctly classified.


In [99]:
# Iterates through emails, classifies them and displays their results.
print("Without updating training data.")
for email in all_emails:
    
    email_information = classify_email(email, previous_spam, previous_ham)
    print(email)
    
    print("Email is spam:", email_information["is_spam"])
    print(f"Email is {email_information['spam']}% likely to be Spam.")
    print(f"Email is {email_information['ham']}% likely to be Ham.")
    print("\n")

        

Without updating training data.
renew your password
Email is spam: True
Email is 73.53% likely to be Spam.
Email is 26.47% likely to be Ham.


renew your vows
Email is spam: False
Email is 49.86% likely to be Spam.
Email is 50.14% likely to be Ham.


benefits of our account
Email is spam: False
Email is 24.15% likely to be Spam.
Email is 75.85% likely to be Ham.


the importance of physical activity
Email is spam: False
Email is 4.39% likely to be Spam.
Email is 95.61% likely to be Ham.




In [100]:
# Iterates through emails, classifies them and displays their results.
print("With updating training data.")
for email in all_emails:
    
    email_information = classify_email(email, training_data["spam"], training_data["ham"])
    print(email)
    
    print("Email is spam:", email_information["is_spam"])
    print(f"Email is {email_information['spam']}% likely to be Spam.")
    print(f"Email is {email_information['ham']}% likely to be Ham.")
    print("\n")
    
    training_data = update_training_data(email, training_data, email_information["is_spam"])

    

With updating training data.
renew your password
Email is spam: True
Email is 73.53% likely to be Spam.
Email is 26.47% likely to be Ham.


renew your vows
Email is spam: True
Email is 51.96% likely to be Spam.
Email is 48.04% likely to be Ham.


benefits of our account
Email is spam: False
Email is 11.01% likely to be Spam.
Email is 88.99% likely to be Ham.


the importance of physical activity
Email is spam: False
Email is 5.13% likely to be Spam.
Email is 94.87% likely to be Ham.




## Reflection

To complete this task, I created the following functions.<br>
**Analyse_emails:** This function accepted a list of emails represented as strings as an argument. It then broke this list of emails into a list of all the words in the emails. These words were set to lowercase to ensure that words were not identified as being unique base on capitalisation. This list of words was then converted to a set to produce a list of unique words. The list of unique words and the list of all words were then used to calculate the probability of occurrence of each word. This was then added to a dictionary with the word as the key and the probability as its value. The total number of words was then added to this dictionary as totat_words to be used in calculations later to classify emails. The function returned a dictionary containing the probability of each word that occurred in the list of emails and the total number of words in the list of emails.<br>
**Prior_probability:** This function calculated P(S) and P(H). It took two lists of emails as arguments and would return the number of emails in the first list over the total number of emails.<br>
**Probability_of_A_given_B:** This function was used to analyse an email as either spam or ham, depending on the training data. It used the analyse_email function to calculate a dictionary of probabilities from the training data. This was then compared to the words in the email to get the likelihood of the email being spam or ham. If a word was not in the training data, Laplace smoothing was used to minimise its effect on the result.<br>
**get_percentage:** Probability_of_A_given_B only calculates the numerator of the Naïve Bayes formula as this is all that is needed to evaluate if an email is spam or ham as the denominator for spam and ham is always equal. However, this resulted in tiny numbers, which didn't inspire me with confidence that my calculations were correct. This function converted the probabilities calculated in Probability_of_A_given_B to a form that is easier to read by adding the denominator and representing the value as a percentage.<br>
**Update_training_data:** Once an email has been classified as spam or ham, this function adds it to the training data as spam or ham.<br>
**Classify_email:** This function uses the functions prior_probability, probability_of_A_given_B, and get_percentage to evaluate an email and classify it as spam or ham. Information about the email is returned as a dictionary containing the probability of the email being spam or ham and a Boolean value that, when True, classifies the email as spam.<br><br>
My code was tested by running all the new emails through my classify_email function. One email was misclassified as ham, which matched my calculations for this task. This was due to the number of unknown words in the email. This is because for the email 'renew your vows', and two words appear in the ham training data while only one appears in the spam training data. <br>
The test was then repeated; however, the training data was updated after each classification. This time all emails were classified correctly. It can be seen that the certainty of an email being spam or ham increases as the test progresses.<br><br>
**Updates based on other student's code**<br>
Another student suggested that I add code that converts the words in the emails being processed to lowercase. This fixed a bug where words would be considered different from each other based on their capitalisation. After this code was added, an email that had previously been classified as spam was classified as ham. Because of this, the word 'Your' in one of the previous ham emails was considered a different word to 'your' in the tested email.<br>
It was also suggested by another student to add stemming to my email classifier by adding a list of words that the classifer could ignore. This is a great idea, and stemming would be of benefit when creating an actual email classifier. However, for this task, I felt there were so few words in the training data that removing words wouldn't provide any benefit.<br><br>
**Problems with this implementation.**<br>
This implementation would need a large set of training data for it to work correctly. When there are many unknown words, an email could get incorrectly classified, which would then add incorrect data t the training data. Another way to fix this is that a user can move emails they consider spam to a spam email folder correcting any mistake made by the algorithm.


<hr>

### Calculations

#### Words in Previous Spam
<table align="left">
  <tr><th>Word</th> <th>Number</th>           <th>Prob Fraction</th>      <th>Prob Decimal</th></tr>
  <tr><td>send</td>   <td>3</td>       <td>3/14</td> <td>0.2142857</td></tr>
  <tr><td>us</td>        <td>2</td>       <td>2/14</td> <td>0.142857</td></tr>
  <tr><td>your</td>   <td>3</td>       <td>3/14</td> <td>0.2142857</td></tr>
  <tr><td>password</td>        <td>2</td>       <td>2/14</td> <td>0.142857</td></tr>
  <tr><td>review</td>        <td>1</td>       <td>1/14</td> <td>0.07142857</td></tr>
  <tr><td>our</td>        <td>1</td>       <td>1/14</td> <td>0.07142857</td></tr>
  <tr><td>website</td>        <td>1</td>       <td>1/14</td> <td>0.07142857</td></tr>
  <tr><td>account</td>        <td>1</td>       <td>1/14</td> <td>0.07142857</td></tr>
  <tr><td>laplace</td>   <td>0</td>       <td>1/16</td> <td>0.0625</td></tr>  
</table>


<hr>

#### Words in Previous Ham
<table align="left">
  <tr><th>Word</th> <th>Number</th>           <th>Prob Fraction</th>      <th>Prob Decimal</th></tr>
  <tr><td>your</td>   <td>1</td>       <td>1/9</td> <td>0.11111</td></tr>
  <tr><td>activity</td>        <td>2</td>       <td>2/9</td> <td>0.22222</td></tr>
  <tr><td>report</td>   <td>1</td>       <td>1/9</td> <td>0.11111</td></tr>
  <tr><td>benifits</td>   <td>1</td>       <td>1/9</td> <td>0.11111</td></tr>
  <tr><td>physical</td>   <td>1</td>       <td>1/9</td> <td>0.11111</td></tr>
  <tr><td>the</td>   <td>1</td>       <td>1/9</td> <td>0.11111</td></tr>
  <tr><td>importance</td>   <td>1</td>       <td>1/9</td> <td>0.11111</td></tr>
  <tr><td>vows</td>   <td>1</td>       <td>1/9</td> <td>0.11111</td></tr>
  <tr><td>laplace</td>   <td>0</td>       <td>1/11</td> <td>0.909091</td></tr>  
</table>


<hr>

#### Spam and Ham Prior Probability

**Spam:** $Spam / (Spam + Ham) = 4 / (4 + 3) = 4/7 = 0.571429$<br>
**Ham:** $Ham / (Spam + Ham) = 3 / (4 + 3) = 3/7 = 0.4285714$<br>

<hr>

#### Email Calculations

'renew your password':<br>
**Spam**<br>
$P(S) * P(renew|S) * P(your|S) * P(password|S)$<br>
$4/7 * 1/16 * 3/14 * 2/14 = 3/2744 = 1.0933e^{-3}$<br><br>

**Ham**<br>
$P(H) * P(renew|H) * P(your|H) * P(password|H)$<br>
$3/7 * 1/11 * 1/9 * 1/11 = 1/2541 = 3.9355e^{-4}$<br><br>

Spam = $P(email|S) / (P(email|S) + P(email|H)) = 1.0933e^{-3}/(1.0933e^{-3} + 3.9355e^{-4}) = 0.7353$<br>
Ham = $P(email|H) / (P(email|S) + P(email|H)) = 3.9355e^{-4}/(1.0933e^{-3} + 3.9355e^{-4}) = 0.2647$<br>
**'renew your password' classified as Spam**<br><br>

<hr>

'renew your vows':<br>
**Spam**<br>
$P(S) * P(renew|S) * P(your|S) * P(vows|S)$<br>
$4/7 * 1/16 * 3/14 * 1/16 = 3/6272 = 4.78316e^{-4}$<br><br>

**Ham**<br>
$P(H) * P(renew|H) * P(your|H) * P(vows|H)$<br>
$3/7 * 1/11 * 1/9 * 1/9 = 1/2079 = 4.81e^{-4}$<br><br>

Spam = $P(email|S) / (P(email|S) + P(email|H)) = 4.78316e^{-4}/(4.78316e^{-4} + 4.81e^{-4}) = 0.4986$<br>
Ham = $P(email|H) / (P(email|S) + P(email|H)) = 4.81e^{-4}/(4.78316e^{-4} + 4.81e^{-4}) = 0.5014$<br>
**'renew your vows' classified as Ham**

<hr>

'benefits of our account':<br>
**Spam**<br>
$P(S) * P(benefits|S) * P(of|S) * P(our|S) * P(account|S)$<br>
$4/7 * 1/16 * 1/16 * 1/14 * 1/14 = 1/87808 = 1.13885e^{-5}$<br><br>

**Ham**<br>
$P(H) * P(renew|H) * P(your|H) * P(vows|H)$<br>
$3/7 * 1/9 * 1/11 * 1/11 * 1/11 = 1/27951 = 3.57769e^{-5}$<br><br>

Spam = $P(email|S) / (P(email|S) + P(email|H)) = 1.13885e^{-5}/(1.13885e^{-5} + 3.57769e^{-5}) = 0.2414$<br>
Ham = $P(email|H) / (P(email|S) + P(email|H)) = 3.57769e^{-5}/(1.13885e^{-5} + 3.57769e^{-5}) = 0.7585$<br>
**'benefits of our account' classified as Ham**<br><br>

<hr>

'the importance of physical activity'<br>
**Spam**<br>
$P(S) * P(the|S) * P(importance|S) * P(of|S) * P(physical|S) * P(activity|S)$<br>
$4/7 * 1/16 * 1/16 * 1/16 * 1/16 * 1/16 = 1/1835008 = 5.44957e^{-7}$<br><br>

**Ham**<br>
$P(H) * P(the|H) * P(importance|H) * P(of|H) * P(physical|H) * P(activity|H)$<br>
$3/7 * 1/9 * 1/9 * 1/11 * 1/9 * 2/9 = 2/168399 = 1.1876555e^{-5}$<br><br>

Spam = $P(email|S) / (P(email|S) + P(email|H)) = 5.44957e^{-7}/(5.44957e^{-7} + 1.1876555e^{-5}) = 0.0439$<br>
Ham = $P(email|H) / (P(email|S) + P(email|H)) = 1.1876555e^{-5}/(5.44957e^{-7} + 1.1876555e^{-5}) = 0.9561$<br>
**'the importance of physical activity' classified as Ham**

<hr>
                    